# Manipulation.py: RRT

In [1]:
import robotic as ry
import manipulation as manip
from maze_utils import generate_maze
from shelf import generate_target_box

In [ ]:
ry.params_add({"rrt/stepsize":.05})

First we need to find a path through the maze. To do this we create an empty config and add an XYPhi joint to which we will attach a block representing the box and robot gripper.

Once this in done we can define an end joint state at the end of the maze. Finally we just call the RRT solver and get the path we need to follow in order to solve the puzzle.

In [15]:
C = ry.Config()
maze_pos = [.0, .3, .69]
start_pos, goal_pos = generate_maze(C, maze_pos)

C.addFrame("base").setPosition([0, 0, start_pos[2]])
C.addFrame("ego", "base") \
	.setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
	.setShape(ry.ST.ssBox, size=[.1, .15, .1, .002]) \
	.setColor([0, 1., 1.]) \
	.setContact(1)

q0 = [*start_pos[:2], 0]
qT = [*goal_pos[:2], 0]

M = manip.ManipulationModelling(C)
M.setup_point_to_point_rrt(q0, qT)
rrt_path = M.solve()

if M.feasible:
	M.play(C, 2)
else:
    print("The RRT solver was unable to find a feasible path.")

Now that we have a path for the robot to follow we can use komo to move the robot arm through the maze.

In [16]:
# Define the new config
del C
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))

start_pos, goal_pos = generate_maze(C, maze_pos)

C.addFrame("box") \
    .setPosition(start_pos) \
    .setShape(ry.ST.ssBox, size=[.03, .03, .03, .001]) \
    .setColor([0., 0., 1.]) \
    .setContact(1) \
    .setMass(1.)

C.delFrame("panda_collCameraWrist")

C.view()

0

In [17]:
# Grab the box and follow the RRT path
M = manip.ManipulationModelling(C)
M.setup_inverse_kinematics()
M.grasp_box(1., "l_gripper", "box", "l_palm", "y")
pose = M.solve()
if M.feasible:
	M.play(C)

	C.attach("l_gripper", "box")
	path_len = len(rrt_path)
	M = manip.ManipulationModelling(C)
	M.setup_multi_phase_komo(path_len-1, accumulated_collisions=False)
	M.follow_path_on_plane_xy(rrt_path, "box")
	path = M.solve()

	if M.feasible:
		M.play(C, 2)
	else:
		M.komo.report(plotOverTime=True)
		print("Path is not feasible!")
		C.view(True)

C.view(True)

  -- feasible:
     { time: 0.001884, evals: 15, done: 1, feasible: 1, sos: 0.0195965, f: 0, ineq: 0.2, eq: 1.78092e-07 }
  -- feasible:
     { time: 0.132527, evals: 73, done: 1, feasible: 1, sos: 0.00998399, f: 0, ineq: 0, eq: 1.28593e-05 }


13

In [18]:
del C

### RRT Sub-motion

In [2]:
# Define the new config
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))

generate_target_box(C, [-.3, .3, 1.])

C.addFrame("box") \
    .setPosition([.3, .3, .8]) \
    .setShape(ry.ST.ssBox, size=[.05, .05, .12, .001]) \
    .setColor([0., 0., 1.]) \
    .setContact(1) \
    .setMass(1.)

C.addFrame("wall") \
    .setPosition([.0, .3, .95]) \
    .setShape(ry.ST.ssBox, size=[.03, .4, .45, .001]) \
    .setColor([0., 0., 0.]) \
    .setContact(1)

# For convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

0

In [3]:
M = manip.ManipulationModelling(C)
M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1)
M.grasp_box(1., gripper, box, "l_palm", "y")
M.place_box(2., box, "target_box_inside", "l_palm", "z")
M.target_relative_xy_position(2., "box", "target_box_inside", [0, 0])
M.solve()

if not M.feasible:
	print("Pick place waypoints infeasible")
else:
	M1 = M.sub_rrt(0)
	M1.solve()
	M2 = M.sub_rrt(1)
	M2.solve()

	if M1.feasible and M2.feasible:
		M1.play(C, 2)
		C.attach("l_gripper", "box")
		M2.play(C, 2)
		C.attach("table", "box")

	else:
		print("No solution found for RRT sub-motion")
	

  -- feasible:
     { time: 0.009493, evals: 18, done: 1, feasible: 1, sos: 0.0560179, f: 0, ineq: 1.02799e-07, eq: 3.3371e-06 }
